## Make grid of study area

In [1]:
import geopandas as gpd
import os
import osgeo  # needed only if running from Windows
import math
import numpy as np
from shapely.geometry import Point, Polygon

In [14]:
# Target grid cell size (in meters)
grid_size = 10000
# Target cell storage (pixel) size (in meters)
cell_size = 10

In [27]:
def grid(bounds, gy, gx, celly, cellx, crs=None):
# from https://github.com/jgrss/zones/blob/master/zones/base.py
    """
    Creates polygon grids

    Args:
        bounds (tuple | BoundingBox): A tuple of (left, bottom, right, top) or a
            ``rasterio.coords.BoundingBox`` instance.
        gy (float): The target grid y size.
        gx (float): The target grid x size.
        celly (float): The y cell size.
        cellx (float): The x cell size.
        crs (Optional[str]): The CRS.

    Returns:
        ``geopandas.GeoDataFrame``
    """

    left, bottom, right, top = bounds

    polys = list()
    nrows = int(abs(top - bottom) / abs(celly))
    print('nrows={}'.format(nrows))
    
    ncols = int(abs(right - left) / abs(cellx))
    print('ncol={}'.format(ncols))

    nrowsp = int(math.ceil((nrows * abs(celly)) / gy))
    print('nrowsp={}'.format(nrowsp))
    ncolsp = int(math.ceil((ncols * abs(cellx)) / gx))
    print('ncolsp={}'.format(ncolsp))

    for i in range(0, nrowsp):
        for j in range(0, ncolsp):
            polys.append(Polygon([(left + j * gx, top - i * gy),
                                  (left + (j * gx) + gx, top - i * gy),
                                  (left + (j * gx) + gx, top - (i * gy) - gy),
                                  (left + j * gx, top - (i * gy) - gy),
                                  (left + j * gx, top - i * gy)]))
    return gpd.GeoDataFrame(data=np.array(list(range(0, len(polys))), dtype='int64'),
                            columns=['grid'],
                            geometry=polys,
                            crs=crs)

In [31]:
df = gpd.read_file(os.path.join(r'data','aois','subAOI_crop_7773.shp'))
left, bottom, right, top = df.total_bounds.tolist()

df_grids = grid((left, bottom, right, top),
                       grid_size, grid_size,
                       cell_size, cell_size,
                       crs=df.crs)

df_grids.loc[:, 'UNQ'] = range(1, df_grids.shape[0]+1)
df_grids.to_file(os.path.join(r'data','gridded_aoi.gpkg'), driver='GPKG')

nrows=13000
ncol=9000
nrowsp=13
ncolsp=9
117
